<h2>Package / SQL Setup</h2>

In [2]:
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('airline_data.db') # automatically creates new file for 
c = conn.cursor()
# When using SQLite, include the following line to ensure foreign key commands are recognized
c.execute('PRAGMA foreign_keys = ON')

t100 = pd.read_csv("t100.csv").dropna(how='all', axis='columns')
db1b_q1 = pd.read_csv("db1b_q1.csv").dropna(how='all', axis='columns')
db1b_q2 = pd.read_csv("db1b_q2.csv").dropna(how='all', axis='columns')
db1b_q3 = pd.read_csv("db1b_q3.csv").dropna(how='all', axis='columns')

db1b = db1b_q1.append(db1b_q2.append(db1b_q3))

assert(len(db1b) == (len(db1b_q1) + len(db1b_q2) + len(db1b_q3)))

db1b.to_sql("db1b",conn,if_exists="replace")
t100.to_sql("t100",conn,if_exists="replace")

<h2>T100 Import and Monthly Scaling Model</h2>

In [3]:
command = '''
SELECT
    DEST, QUARTER, MONTH, SUM(PASSENGERS)
FROM
    t100
WHERE
    QUARTER != 4
GROUP BY
    DEST, QUARTER, MONTH
'''

t100_data = pd.read_sql_query(command,conn)

sorted_t100 = t100_data.groupby(["DEST","QUARTER","MONTH"]).first()
state_pcts = sorted_t100.groupby(level=[0,1]).apply(lambda x: x / float(x.sum()))
state_pcts.reset_index(inplace=True)

<h2>DB1B Import and T100 Join</h2>

In [52]:
command = '''
SELECT
    ORIGIN, ORIGIN_STATE_ABR, DEST, DEST_STATE_ABR, QUARTER, 10*SUM(PASSENGERS) as total_pax
FROM
    db1b
GROUP BY
    QUARTER, ORIGIN, DEST
'''

db1b_import = pd.read_sql_query(command,conn)
merged_df = db1b_import.merge(state_pcts, on = ["DEST", "QUARTER"], how = "left")
merged_df = merged_df.rename(columns = {'SUM(PASSENGERS)':'pax_ratio'})
merged_df['monthly_pax'] = merged_df['total_pax'] * merged_df['pax_ratio']
cleaned_table = merged_df.drop(columns = ['QUARTER','total_pax','pax_ratio'])




<h2>Grouping by State</h2>

In [63]:
state_table = cleaned_table.groupby(["ORIGIN_STATE_ABR","DEST_STATE_ABR","MONTH"]).sum().dropna(how='all', axis='rows')

#example for one state to state pair
state_table.loc['RI'].loc['WY']

,monthly_pax
MONTH,
1.0,117.225021
2.0,143.498203
3.0,69.276776
4.0,0.845070
5.0,3.887324
6.0,5.267606
7.0,73.108275
8.0,77.983213
9.0,68.908512


<h3>Ideas from 2/24</h3>

* t100 carries every individual NONSTOP route's passengers per month. so no info on multi-leg flights
* on the other hand, db1b only has quarterly data for every itinerary -- so we can look at multi-leg flights and specifies the exact route (i.e. PVD->BWI->MCO)
* idea: using t100 to calculate "scaling factor" for a given route that we both know (BOS->MCO nonstop defined in both) so we can estimate monthly passengers for any route. More or less adding a month field to db1b by estimation
* we decided this b/c better approximation than neglecting all flights with stops, which we would have to do if t100 was our primary data source
* probably no need to use on-time data

what's left?

* figure out monthly scaling factor / add column to db1b
* which routes do we want to keep
* once we do this and have monthly idea of every city -> city, we need to group by state

ideas

* scaling state by state? it would be nonstops only in t100 for state-state, so we wouldn't be able to scale for every state-state pair. won't work.
* possible solution: scale non-stop to non-stops that we have, and get an idea of the data. Is st dev low / are they pretty similar? If so, we can broadly apply to everything. otherwise, 
* new (best?) idea: scale by origin or destination airport. calculate monthly distribution for one airport, and then apply that to every flight involved at that airport, regardless of market coupon. assumes that there's no difference in monthly distribution as a function of number of stops, but that's probably fair